In [ ]:
import numpy as np
import pickle
import pandas as pd 
from scipy import spatial
from sklearn.neighbors import KDTree, BallTree
pd.set_option('display.max_columns', None)

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler
from scipy.spatial import distance
import matplotlib.pyplot as plt                   # For graphics
import seaborn as sns

In [ ]:
#df_comp_nona = pd.read_pickle("./OUT_dfs/df_comp_full_processed_nona.pkl")
#df_c = df_comp_nona.copy()

df_appr_nona = pd.read_pickle("./OUT_dfs/df_appr_full_processed_nona.pkl")#
df_a = df_appr_nona.copy()

dict_apprid_to_uniquecompidnew = pickle.load(open("./OUT_dfs/dict_apprid_to_uniquecompidnew", "rb"))
uniqueComps = pd.read_pickle("./OUT_dfs/uniqueComps.pkl")

In [ ]:
uniqueComps.columns

In [ ]:
test_appr = 857133
test_comps_newids = dict_apprid_to_uniquecompidnew[test_appr]

# Calculate distance in time and space from test row

In [ ]:
def calculate_time_and_space_difference(chosen_appr_id, appr_dataframe, comp_dataframe, minimum_days_from_appr, distance_measure="cityblock"):
    test_row = appr_dataframe[appr_dataframe["SUBJ_APPR_ID"]==chosen_appr_id]
    #time difference
    test_row["DATEDIFF"] = -minimum_days_from_appr
    comp_dataframe["DATEDIFF"] = comp_dataframe["SALEDATE"] - test_row["SALEDATE"].values[0]
    comp_dataframe["DATEDIFF"] = comp_dataframe["DATEDIFF"]/ np.timedelta64(1, 'D')
    
    #space difference
    test_row["DISTANCEDIFF"]=0
    test_row_dist = np.array([test_row["APPRLATITUDE"].values[0],test_row["APPRLONGITUDE"].values[0]])
    comp_dataframe["DISTANCEDIFF"] = distance.cdist(np.array(comp_dataframe[["APPRLATITUDE","APPRLONGITUDE"]]),
                                           test_row_dist[None,],
                                          distance_measure)
    return comp_dataframe, test_row

In [ ]:
uniqueComps2, y2 = calculate_time_and_space_difference(chosen_appr_id=test_appr, 
                                       appr_dataframe=df_a, 
                                       comp_dataframe=uniqueComps, 
                                       minimum_days_from_appr=60,
                                                distance_measure="cityblock")

In [ ]:
# Calculate significant correlations
cm = uniqueComps2.drop(['ZIPCODE', 'STATE',"SALEDATE","COMPSALEDATE","COUNTY","CITY","ADDRESS1",
                                                  "UNIQUECOMPIDNEW","COMPNUM","SUBJ_APPR_ID", ], axis=1).corr().unstack().sort_values().drop_duplicates()

In [ ]:
cm.head(10)

In [ ]:
cm.tail(25)